In [ ]:
from pandas import Series
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import plot_confusion_matrix
from scipy.stats import kruskal

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_csv('/kaggle/input/sf-dst-scoring/train.csv')
test_df = pd.read_csv('/kaggle/input/sf-dst-scoring/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

Заполняем пропуски признака education, используя распределение возможных значений из train выборки

In [ ]:
edu_levels = train_df['education'].dropna().unique()
distributions = train_df['education'].value_counts(normalize=True)
missing_train = train_df['education'].isnull()
train_df.loc[missing_train, 'education'] = np.random.choice(distributions.index, size=len(train_df[missing_train]),p=distributions.values)
missing_test = test_df['education'].isnull()
test_df.loc[missing_test, 'education'] = np.random.choice(distributions.index, size=len(test_df[missing_test]),p=distributions.values)

client_id - идентификатор клиента

education - уровень образования

sex - пол заемщика

age - возраст заемщика

car - флаг наличия автомобиля

car_type - флаг автомобиля иномарки

decline_app_cnt - количество отказанных прошлых заявок

good_work - флаг наличия “хорошей” работы

bki_request_cnt - количество запросов в БКИ

home_address - категоризатор домашнего адреса

work_address - категоризатор рабочего адреса

income - доход заемщика

foreign_passport - наличие загранпаспорта

sna - связь заемщика с клиентами банка

first_time - давность наличия информации о заемщике

score_bki - скоринговый балл по данным из БКИ

region_rating - рейтинг региона

app_date - дата подачи заявки

default - флаг дефолта по кредиту

Выявляем численные / категориальные факторы

In [ ]:
for c in train_df.columns:
    print(f"{c} -> {train_df[c].nunique()}")

In [ ]:
num_cols = ['age', 'income', 'region_rating', 'score_bki']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = ['education', 'home_address', 'work_address', 'first_time', 'sna', 'decline_app_cnt', 'bki_request_cnt']

Изучаем распределение численных признаков

In [ ]:
rows = len(num_cols) / 2 + 1
ncols = 2
fig = plt.figure(figsize=(17, 20))
for i,c in enumerate(num_cols):
    ax = fig.add_subplot(rows, ncols, i + 1)
    train_df[c].hist(ax = ax,bins=100)
    ax.set_xlabel(c)

score_bki распределен нормально, остальные численные признаки смещены в разной степени, попробуем нормализовать их

In [ ]:
def log_for_col(data):
    return data.apply(lambda v: np.log(v + 1))

In [ ]:
rows = len(num_cols) / 2 + 1
ncols = 2
fig = plt.figure(figsize=(17, 20))
for i,c in enumerate(num_cols):
    ax = fig.add_subplot(rows, ncols, i + 1)
    log_for_col(train_df[c]).hist(ax = ax,bins=100)
    ax.set_xlabel(c)

Самая значительная корректировка распределения заметна для признака income, применим логарифмирование к его значениям

In [ ]:
train_df['income'] = log_for_col(train_df.income)
test_df['income'] = log_for_col(test_df.income)

Оценим сбалансированность классов default == 0 и default == 1 в обучающей выборке

In [ ]:
train_df.default.value_counts(normalize=True)

Кол-во обучающих примеров плохих заемщиков значительно меньше кол-ва примеров хороших заемщиков. Нужно использовать балансировку классов при построении модели.

Оценим влияние факторов на целевую переменную

In [ ]:
imp_num = Series(f_classif(train_df[num_cols], train_df['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
label_encoder = LabelEncoder()
for column in bin_cols:
    train_df[column] = label_encoder.fit_transform(train_df[column])
    test_df[column] = label_encoder.fit_transform(test_df[column])
train_df['education'] = label_encoder.fit_transform(train_df['education'])
test_df['education'] = label_encoder.fit_transform(test_df['education'])

In [ ]:
imp_cat = Series(mutual_info_classif(train_df[bin_cols + cat_cols], train_df['default'], discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Исследуем взаимное влияние факторов друг на друга

In [ ]:
plt.subplots(figsize=(20,15))
sns.heatmap(train_df[num_cols + cat_cols + bin_cols].corr().abs(), vmin=0, vmax=1)

* Заметна связь между уровнем дохода и наличием загран. паспорта, для других численных факторов перекрестных связей не найдено
* Заметна умеренная связь уровня дохода с наличием и типом транспортного средства, а так же с полом потенциального заемщика
* Заметна связь между полом заемщика и наличием машины
* Так же вилим сильную связь между наличием машины и ее типом, что довольно естественно, т.к. если car_type == Y, то и car == Y
* заметна связь между давностью наличия информации и связью заемщика с клиентами банка
* заметна сильная связь между метами жительства и работы

Преобразуем факторы train датасета - категориальные переменные закодируем с помощью one-hot encoding'а, численные факторы стандартизируем

In [ ]:
hot_encoder = OneHotEncoder(sparse = False)
hot_encoder.fit(pd.concat([train_df[cat_cols], test_df[cat_cols]]).values)
def encode_factors(df):
    X_cat = pd.DataFrame(hot_encoder.transform(df[cat_cols].values))
    X_num = StandardScaler().fit_transform(df[num_cols].values)
    X_bin = df[bin_cols].values
    return np.hstack([X_num, X_bin, X_cat])

In [ ]:
X = encode_factors(train_df)
Y = train_df['default'].values
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=100500)

train_index, test_index = list(splitter.split(X, Y))[0]
X_train, X_test = X[train_index], X[test_index]
Y_train, Y_test = Y[train_index], Y[test_index]

Запускаем подбор коэффициента регуляризации и решателя задачи минимизации:
* коэффициент L2 регуляризации будем выбирать из пространства десятичных логарифмов от 0 до 2
* будем тестировать: модифицированный метод Ньютона и метод Бройдена-Флетчера-Гольдфарба-Шанно

Используем ballanced class_weight и фиксированную отсечку максимального кол-ва итераций обучения в 1000.
Подбирать будем на 7 разбиениях, оптимизируя f1 score.

In [ ]:
from sklearn.model_selection import GridSearchCV
#C=np.linspace(0.01, 50)
C = np.logspace(0, 2)
hyperparameters = dict(C=C, penalty=['l2'], solver=['newton-cg','lbfgs'], class_weight=['balanced'], max_iter=[1000])
model = LogisticRegression()
clf = GridSearchCV(model, hyperparameters, cv=7, verbose=0, scoring='f1')
best_model = clf.fit(X_train, Y_train)
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])
probs = best_model.predict_proba(X_test)
probs = probs[:,1]
fpr, tpr, threshold = roc_curve(Y_test, probs)
roc_auc = roc_auc_score(Y_test, probs)
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.6f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()
best_model.best_estimator_

Оценим confusion matrix для полученной модели

In [ ]:
plot_confusion_matrix(best_model.best_estimator_, X_test, Y_test)
plt.show()

Можно заметить, что модель корректно классифицирует буольшую часть примеров

Применим аналогичное кодирование к test датасету и вычислим вероятности невозврата долгов для представленных в нем заемщиков

In [ ]:
X_target = encode_factors(test_df)
target_prob = best_model.best_estimator_.predict_proba(X_target)[:,1]

Сохраняем результат

In [ ]:
result = pd.concat([test_df.client_id, Series(target_prob, name='default')], axis=1)
result.head()

In [ ]:
result.to_csv('submission.csv', index=False)